In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
base_model = AutoModelForCausalLM.from_pretrained(
    "bigcode/starcoder2-3b",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "lora_weight")
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")


d:\UET_2025\DL\BTL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [3]:

prompt = r'''
Given a list of integers `nums` of size `n`, return the sum of the two largest integers.

Example:
Input: `nums = [1, 2, 3, 4, 5]`
Output: `9`

Explanation:
The two largest integers are 4 and 5, so the sum is 9.

Constraints:
- `n < 10000`


Boilerplate code:
```python
def sum_of_two_largest(nums):
    ...
```

'''


In [4]:
formatted_prompt = f"""### Problem:
{prompt.strip()}

Instruction: Write exactly one clean and efficient Python function that correctly solves the problem.
The solution should be optimal in both time and space complexity. 
Do not include multiple solutions or explanations.

### Solution:
"""

In [ ]:
from transformers import GenerationConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Tokenize
inputs = tokenizer(
    formatted_prompt,
    return_tensors="pt",
    padding=False,
    truncation=True,
    max_length=2048
).to(device)

# Cấu hình sinh text
generation_config = GenerationConfig(
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# Sinh
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        generation_config=generation_config
    )

# Giải mã
full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Chỉ lấy phần sau "### Solution:"
generated_code = full_output.split("### Solution:")[-1].strip()
generated_code = generated_code.split("\n\n<|end_of_solution|>")[0].strip()

print("=== Generated Code ===")
print(generated_code)

=== Generated Code ===
def sum_of_two_largest(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    largest = second_largest = -math.inf
    for num in nums:
        if num > largest:
            second_largest = largest
            largest = num
        elif num > second_largest:
            second_largest = num
    return largest + second_largest
